In [ ]:
# from scipy.cluster import  hierarchy

# #Vectorizing
# X = CountVectorizer().fit_transform(docs)
# X = TfidfTransformer().fit_transform(X)
# #Clustering
# X = X.todense()
# threshold = 0.1
# Z = hierarchy.linkage(X,"average", metric="cosine")
# C = hierarchy.fcluster(Z, threshold, criterion="distance")

In [1]:
from deeppavlov.core.common.file import read_json
from deeppavlov import build_model, configs
from sklearn.cluster import KMeans
from os import listdir
from os.path import isfile, join
from tqdm.notebook import tqdm
import re
import nltk
import pickle
import swifter

In [2]:
bert_config = read_json(configs.embedder.bert_embedder)
bert_config['metadata']['variables']['BERT_PATH'] = 'retrained_bert_pythorch'

m = build_model(bert_config)

dramas = [f for f in listdir('drama/') if isfile(join('drama/', f))]
dramas_texts = []


for drama in dramas:
    with open(join('drama/', drama), 'rb') as f:
        dramas_texts.append(pickle.load(f))

dramas_texts[0][0][:5]

[nltk_data] Downloading package punkt to /Users/vlasovvp/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vlasovvp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /Users/vlasovvp/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /Users/vlasovvp/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


[('Григорий', 'Ступай отвори дверь! звонят!'),
 ('Петр',
  'Да ты что сидишь? На ногах у тебя пузыри, что ли? встать не можешь?'),
 ('Иван', 'Ну, уж я пойду, так и быть, отворю! Это Андрюшка!'),
 ('Григорий', 'А, московская ворона! Откуда тебя принесло?'),
 ('Чужой слуга',
  'Ах ты, чухонский сын! Побегал бы ты с мое. Вот к цветочнице велела снесть, что на Петербургской. Небось четвертака на извозчика не\n            даст. Да и к вашему тож. Что, спит?')]

In [4]:
only_sents = []

for drama in tqdm(dramas_texts):
    for part in drama:
        only_sents = only_sents + [sent for _, sent in part]
        
only_sents[:5]

['Ступай отвори дверь! звонят!',
 'Да ты что сидишь? На ногах у тебя пузыри, что ли? встать не можешь?',
 'Ну, уж я пойду, так и быть, отворю! Это Андрюшка!',
 'А, московская ворона! Откуда тебя принесло?',
 'Ах ты, чухонский сын! Побегал бы ты с мое. Вот к цветочнице велела снесть, что на Петербургской. Небось четвертака на извозчика не\n            даст. Да и к вашему тож. Что, спит?']

In [6]:
only_sents_splited = []

for text in tqdm(only_sents[:10000]):
    only_sents_splited = only_sents_splited + nltk.sent_tokenize(re.sub('[\n]', ' ', text))
    
only_sents_splited[:5]

['Ступай отвори дверь!',
 'звонят!',
 'Да ты что сидишь?',
 'На ногах у тебя пузыри, что ли?',
 'встать не можешь?']

In [8]:
only_sents_vectors_pooler = []
sents_count = len(only_sents_splited)

for i in tqdm(range(int(sents_count / 5))):
    _, _, _, _, _, only_sents_vectors_mean_tmp, bert_pooler_outputs_tmp = m([only_sents_splited[i * 5: (i+1)*5]])
    only_sents_vectors_pooler.extend(bert_pooler_outputs_tmp)
    
with open('ru_bert_fullsplited_vectors.pkl', 'wb') as f:
    pickle.dump(zip(only_sents_splited, only_sents_vectors_pooler), f)
    
only_sents_vectors_pooler[:1]

[array([-2.65993923e-01,  3.72807384e-01,  7.51957536e-01,  8.14951241e-01,
        -1.71166495e-03, -3.20786446e-01, -2.31028154e-01, -4.34515566e-01,
        -8.12997341e-01, -2.90729821e-01,  1.36019364e-01, -1.87365487e-01,
        -5.33657730e-01, -2.87924021e-01,  3.19625318e-01, -7.61417329e-01,
         2.82152027e-01,  6.05236351e-01, -1.75235137e-01,  2.18629047e-01,
        -6.54691517e-01,  8.19572866e-01,  5.15520275e-01,  2.19205081e-01,
         2.27224126e-01,  8.15517679e-02, -2.10639849e-01, -2.22462595e-01,
         7.02796817e-01,  4.41654205e-01, -4.75546896e-01, -4.65574890e-01,
        -7.09393620e-03,  3.40478301e-01,  6.73024952e-01, -3.98593873e-01,
        -3.60574603e-01, -3.18617048e-03,  8.34443048e-02,  4.36448485e-01,
         1.02244213e-01,  6.88900888e-01,  3.68929714e-01, -5.35135806e-01,
        -1.55092403e-01, -4.28882539e-01, -7.93013871e-01,  4.30650413e-02,
         1.88734770e-01,  1.78700700e-01,  4.39796418e-01, -5.61181903e-01,
         4.0

In [14]:
with open('ru_bert_fullsplited_vectors.pkl', 'rb') as f:
    sents_vectors = pickle.load(f)

In [13]:
only_sents_vectors_pooler = []
for a in tqdm(sents_vectors):
    only_sents_vectors_pooler.append(a[1])

In [10]:
sents_vectors = list(zip(only_sents_splited, only_sents_vectors_pooler))
sents_vectors[:1]

[('Ступай отвори дверь!',
  array([-2.65993923e-01,  3.72807384e-01,  7.51957536e-01,  8.14951241e-01,
         -1.71166495e-03, -3.20786446e-01, -2.31028154e-01, -4.34515566e-01,
         -8.12997341e-01, -2.90729821e-01,  1.36019364e-01, -1.87365487e-01,
         -5.33657730e-01, -2.87924021e-01,  3.19625318e-01, -7.61417329e-01,
          2.82152027e-01,  6.05236351e-01, -1.75235137e-01,  2.18629047e-01,
         -6.54691517e-01,  8.19572866e-01,  5.15520275e-01,  2.19205081e-01,
          2.27224126e-01,  8.15517679e-02, -2.10639849e-01, -2.22462595e-01,
          7.02796817e-01,  4.41654205e-01, -4.75546896e-01, -4.65574890e-01,
         -7.09393620e-03,  3.40478301e-01,  6.73024952e-01, -3.98593873e-01,
         -3.60574603e-01, -3.18617048e-03,  8.34443048e-02,  4.36448485e-01,
          1.02244213e-01,  6.88900888e-01,  3.68929714e-01, -5.35135806e-01,
         -1.55092403e-01, -4.28882539e-01, -7.93013871e-01,  4.30650413e-02,
          1.88734770e-01,  1.78700700e-01,  4.3979

In [10]:
clust_model_kmeans = KMeans(n_clusters=500, random_state=42)
clust_model_kmeans.fit(only_sents_vectors_pooler)
clusters = clust_model_kmeans.predict(only_sents_vectors_pooler)
clusters[:5]

array([488, 457, 182, 186, 179], dtype=int32)

In [16]:
sents_with_clusters = list(zip([sent for sent, _ in sents_vectors], clusters))

In [17]:
for curr_cluster in sorted(list(set(clusters))):
    print(curr_cluster)
    sents_in_this_cluster = [sent for sent, cluster in sents_with_clusters if cluster==curr_cluster]
    for sent in sents_in_this_cluster[:10]:
        print(sent)
    print('=========================\n=========================\n=========================\n\n\n')

0
Сыновья-то там запрутся, да и пьют, кранболь               у них там всякий идет.
А!
Я ведь знаю, у вас один разговор: по любви выйти               замуж.
Бог с тобой,               что ты это!
Не сердитесь на меня, Марья Андревна.
Я вам желаю всякого счастья.
Дарья подает на подносе бутылку и рюмки и ставит на стол.



1
А он вас любит, вы думаете?
Ты, Маша, согласна?



2
(Цензор изменил: вместо славянского «живота своего» поставил               «жизни своей».
Вы, кажется, начали отнекиваться, Лизавета Платоновна?
Ах!
Может быть, теперь именно мне?!
Уж положитесь на меня.
Странно, я говорю ему, Миша, ты человек               образованный, а такой застенчивый; чего тебе, друг мой, с твоим умом и образованием               бояться?
-- "Но зачем же навсегда,               Иван Иваныч??
А хорошо ли она образована?
Да уж кроме Максима Дорофеича, некого просить.
Тебе меня только расстроить хочется.



3
Мосинька             моя!..
Кровь-то пустить надобно ему, а не ей.
Не сон ли я вижу!


Слышите?..
Вот вопрос!
Ах!
У вас несколько             нижних пуговиц не застегнуто.
Пожалуй… пожалуй… сочиним!
Почему же разогреты?
!
Не перебивать!..
Аграфена Панкратьевна идет!..
Занимаем —



275
Откуда тебя принесло?
О!
Е…
Чего я для него не сделаю!..
Аль другое написать?



276
Нашел!..
Почему это знати другому?
Да разве на колокольнях-то звонят по-латыни?
Уж и до этого-то горя я не знала, что делать, а теперь вовсе дура сделалась.
Я уж к вам               посылала Дарью.
И я               чувствую, что во мне есть силы.
Хорошо-с, теперь понимаю.
Вот-с, понравился наш Перекузов отцу;             понравился и дочери... Кажись, за чем бы дело стало?



277
И как они, пришед в убожество,             приемыша своего из дому маленького отдали другим, которые его не приемышем, да только из             одного человеколюбия возрастили и, возрастив, отпустили его на собственное пропитание и             которые также допра-шиваны, как и та кормилица, которая его кормила.



278
Ступай отво

In [18]:
def cluster_text(per_text, bert_model, cluster_model):
    for person, text in per_text:
        sents = {sent: -1 for sent in nltk.sent_tokenize(re.sub('[\n]', ' ', text))}
        for sent in sents:
            _, _, _, _, _, _, pooler_vectors = m([sent])
            cluster = cluster_model.predict(pooler_vectors)
            sents[sent] = cluster[0]
        print('{}: {}'.format(person, sents))
#     return text

cluster_text(dramas_texts[4][0], m, clust_model_kmeans)

Шааф: {'Ф червёх.': 101}
Анна Семеновна: {'Опять?': 269, 'Да ты нас, батюшка, эдак совсем заиграешь.': 212}
Шааф: {'Фоземь ф червёх.': 475}
Анна Семеновна: {'Каков!': 181, 'С ним играть нельзя.': 113}
Наталья Петровна: {'Что ж вы перестали?': 310, 'Читайте.': 356}
Ракитин: {'«Monte-Cristo se redressa haletant...»«Монте-Кристо вскочил,               прерывисто дыша...» (Франц.': 466, ')(«Монте-Кристо вскочил,               прерывисто дыша...» (Франц.))': 382, 'Наталья Петровна, вас это занимает?': 40}
Наталья Петровна: {'Нисколько.': 246}
Ракитин: {'Для чего же мы читаем?': 148}
Наталья Петровна: {'А вот для чего.': 450, 'На днях мне одна дама говорила: «Вы не читали „Монте-Кристо“?': 299, 'Ах,             прочтите — это прелесть».': 339, 'Я ничего ей не отвечала тогда, а теперь могу ей сказать, что             читала и никакой прелести не нашла.': 144}
Ракитин: {'Ну да если вы теперь уже успели убедиться...': 339}
Наталья Петровна: {'Ах, какой вы ленивый!': 40}
Ракитин: {'Я готов, поми

Ракитин: {'Чужая душа — темный лес.': 246, 'Но к чему эти намеки... За что вы меня то и дело колете?': 382}
Наталья Петровна: {'Кого же колоть, коли не друзей... А вы мой друг... Вы это знаете.': 273, 'Вы мой старый друг.': 246}
Ракитин: {'Боюсь я только... как бы этот старый друг вам не приелся...': 247}
Наталья Петровна: {'Одни хорошие вещи приедаются.': 371}
Ракитин: {'Может быть... Только от этого им не легче.': 273}
Наталья Петровна: {'Полноте...  Как будто вы не знаете... ce que vous êtes             pour moiчто́ вы для меня (франц.).': 456, '(что́ вы для меня (франц.).': 339, ').': 269}
Ракитин: {'Наталья Петровна, вы играете со мной, как кошка с мышью... Но мышь не жалуется.': 339}
Наталья Петровна: {'О, бедный мышонок!': 460}
Анна Семеновна: {'Двадцать с вас, Адам Иваныч... Ага!': 181}
Шааф: {'Я фперет Лисафет Богдановне не приклашаю.': 148}
Матвей: {'Игнатий Ильич приехали-с.': 113}
Шпигельский: {'Об докторах не докладывают.': 246, 'Нижайшее мое почтенье всему             сем

Наталья Петровна: {'Мои намерения?': 283}
Шпигельский: {'То есть... говоря без обиняков, мой знакомый...': 97}
Наталья Петровна: {'Уж не сватается ли за нее?': 246}
Шпигельский: {'Точно так-с.': 246}
Наталья Петровна: {'Вы шутите?': 40}
Шпигельский: {'Никак нет-с.': 246}
Наталья Петровна: {'Да помилуйте, она еще ребенок; какое странное поручение!': 250}
Шпигельский: {'Чем же странное, Наталья Петровна?': 40, 'мой знакомый...': 269}
Наталья Петровна: {'Вы большой делец, Шпигельский... А кто такой ваш знакомый?': 212}
Шпигельский: {'Позвольте, позвольте.': 101, 'Вы мне еще ничего не сказали положительного насчет...': 273}
Наталья Петровна: {'Полноте, доктор.': 246, 'Вера еще дитя.': 101, 'Вы сами это знаете, господин дипломат.': 437, 'Да вот, кстати, и она.': 40}
Коля: {'Ракитин, вели нам клею дать, клею...': 246}
Наталья Петровна: {'Откуда вы?': 356, 'Как ты раскраснелась...': 247}
Вера: {'Из саду...  Здравствуйте, Игнатий Ильич.': 247}
Ракитин: {'На что тебе клею?': 356}
Коля: {'Нужно,

Ислаев: {'Этого... так сказать... этой любви к работе нету... именно любви нет.': 382, 'Он тебе мненья             твоего хорошо высказать не даст, — «Слушаю, батюшка...» А какое: слушаю — просто ничего             не понял.': 60, 'Посмотри-ка на немца — то ли дело!': 40, 'Терпенья у русского нет.': 460, '— Со всем тем, я             его уважаю... А где Наташа?': 212, 'Не знаешь?': 356}
Ракитин: {'Она сейчас здесь была.': 460}
Ислаев: {'Да который час?': 201, 'Пора бы обедать.': 246, 'С утра на ногах — дела пропасть... А еще сегодня на             постройке не был.': 212, 'Время так вот и уходит.': 371, 'Беда!': 269, '— просто никуда не поспеваешь!': 40, 'Ты, я вижу, смеешься надо мной... Да что ж, брат,             делать?': 212, 'Кому что.': 269, 'Я человек положительный, рожден быть хозяином — и больше ничем.': 212, 'Было             время — я о другом мечтал; да осекся, брат!': 475, 'Пальцы себе обжег — во-как!': 224, '— Что это Беляев             не идет?': 380}
Ракитин: {'Кто так

Наталья Петровна: {'Что такое?': 431, 'Кто упал?': 418}
Вера: {'Нет-с... там качели Алексей Николаич устроил, так нянюшка вот вздумала...': 339}
Наталья Петровна: {'Ах, кстати, Шпигельский, подите-ка сюда...  Она не ушиблась?': 135}
Вера: {'О нет-с!': 246}
Наталья Петровна: {'Да... а всё-таки, Алексей Николаич, это вы напрасно...': 339}
Матвей: {'Кушанье готово-с.': 246}
Наталья Петровна: {'А!': 356, 'Да где ж Аркадий Сергеич?': 250, 'Вот они опять опоздают с Михайлом Александровичем.': 246}
Матвей: {'Они уж в столовой-с.': 246}
Наталья Петровна: {'А маменька?': 356}
Матвей: {'В столовой и онэ-с.': 246}
Наталья Петровна: {'А!': 356, 'ну, так пойдемте.': 460, 'Вера, allez en avant avec               monsieurидите вперед с месье (франц.).': 339, '(идите вперед с месье (франц.).)': 475}
Шпигельский: {'Вы мне что-то хотели сказать?': 246}
Наталья Петровна: {'Ах, да!': 283, 'Точно... Вот видите ли... Мы еще с вами поговорим о... о вашем предложенье.': 382}
Шпигельский: {'Насчет... Веры Алек

In [ ]:
len(only_sents_vectors_mean[0])

In [ ]:
help(pickle.dump)

In [18]:
import enum
print(enum.__file__)  

/opt/anaconda3/lib/python3.7/enum.py


In [73]:
from sklearn.metrics.pairwise import pairwise_distances

distance_matrix = pairwise_distances(only_sents_vectors_pooler, 
                                 only_sents_vectors_pooler, metric='cosine')
    
distance_matrix

array([[2.38418579e-07, 1.69472098e-02, 1.43091679e-02, ...,
        2.58750319e-02, 2.37394571e-02, 3.43723297e-02],
       [1.69472098e-02, 0.00000000e+00, 1.28476620e-02, ...,
        2.13153362e-02, 2.13551521e-02, 3.24190855e-02],
       [1.43091679e-02, 1.28476620e-02, 0.00000000e+00, ...,
        1.34688616e-02, 1.21254325e-02, 2.31278539e-02],
       ...,
       [2.58750319e-02, 2.13153362e-02, 1.34688616e-02, ...,
        0.00000000e+00, 1.05650425e-02, 1.12242699e-02],
       [2.37394571e-02, 2.13551521e-02, 1.21254325e-02, ...,
        1.05650425e-02, 0.00000000e+00, 9.76693630e-03],
       [3.43723297e-02, 3.24190855e-02, 2.31278539e-02, ...,
        1.12242699e-02, 9.76693630e-03, 1.19209290e-07]], dtype=float32)

In [81]:
import hdbscan
import numpy as np
from sklearn.metrics.pairwise import cosine_distances

def hdbscan_cluster_text(per_text, bert_model, cluster_model):
    for person, text in per_text:
        sents = {sent: -1 for sent in nltk.sent_tokenize(re.sub('[\n]', ' ', text))}
        for sent in sents:
            _, _, _, _, _, _, pooler_vectors = m([sent])
            cluster, _ = hdbscan.approximate_predict(cluster_model, pooler_vectors)
            sents[sent] = cluster[0]
        print('{}: {}'.format(person, sents))

clusterer_2 = hdbscan.HDBSCAN(min_cluster_size=2, gen_min_span_tree=True,
                              metric='precomputed')
clusters = clusterer_2.fit_predict(np.array(distance_matrix).astype('double'))

In [75]:
hdbscan.approximate_predict(clusterer_2, only_sents_vectors_pooler[:1])

AttributeError: No prediction data was generated

In [55]:
hdbscan_cluster_text(dramas_texts[0][2], m, clusterer)

AttributeError: No prediction data was generated

In [37]:
np.array(only_sents_vectors_pooler).astype('double')

array([[-0.26599392,  0.37280738,  0.75195754, ..., -0.08716362,
         0.01412724, -0.6184088 ],
       [-0.36476564,  0.37615857,  0.80821925, ..., -0.25867048,
         0.01337025, -0.66811579],
       [-0.22228356,  0.4503465 ,  0.82153881, ..., -0.17645346,
         0.10380512, -0.65037233],
       ...,
       [-0.22423857,  0.54263043,  0.78240395, ..., -0.26817462,
         0.1560757 , -0.74449301],
       [-0.25514507,  0.44760236,  0.77914333, ..., -0.28813988,
         0.1676462 , -0.65102547],
       [-0.33731803,  0.50935203,  0.72115499, ..., -0.27596992,
         0.0896984 , -0.69960111]])

In [20]:
help(hdbscan.HDBSCAN.metric)

Help on class HDBSCAN in module hdbscan.hdbscan_:

class HDBSCAN(sklearn.base.BaseEstimator, sklearn.base.ClusterMixin)
 |  HDBSCAN(min_cluster_size=5, min_samples=None, cluster_selection_epsilon=0.0, metric='euclidean', alpha=1.0, p=None, algorithm='best', leaf_size=40, memory=Memory(location=None), approx_min_span_tree=True, gen_min_span_tree=False, core_dist_n_jobs=4, cluster_selection_method='eom', allow_single_cluster=False, prediction_data=False, match_reference_implementation=False, **kwargs)
 |  
 |  Perform HDBSCAN clustering from vector array or distance matrix.
 |  
 |  HDBSCAN - Hierarchical Density-Based Spatial Clustering of Applications
 |  with Noise. Performs DBSCAN over varying epsilon values and integrates
 |  the result to find a clustering that gives the best stability over epsilon.
 |  This allows HDBSCAN to find clusters of varying densities (unlike DBSCAN),
 |  and be more robust to parameter selection.
 |  
 |  Parameters
 |  ----------
 |  min_cluster_size : i

In [30]:
from sklearn.metrics.pairwise import pairwise_distances

In [66]:
pairwise_distances([[1,2]], [[1, 1.5]], metric='cosine')[0][0]

0.007722123286332372

In [86]:
clusterer_2.single_linkage_tree_.plot(cmap='viridis', colorbar=True)

RecursionError: maximum recursion depth exceeded in comparison